# 역세권 건물 EDA

이 노트북은 `Station_Catchment_Buildings` 테이블의 데이터를 탐색하고 시각화합니다.
역세권 내 건물의 용도, 구조, 높이, 연면적 등의 특성을 분석합니다.

In [ ]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from huggingface_hub import hf_hub_download

# 설정
repo_id = "alrq/subway"       # 데이터셋 리포지토리 ID
filename = "db/subway.db"     # 리포지토리 내 파일 경로
local_dir = "../../data"               # 다운로드 받을 로컬 기본 경로 (이 경우 ./db/subway.db 로 저장됨)
# 파일 다운로드
# local_dir을 지정하면 리포지토리의 폴더 구조를 유지하며 파일을 저장합니다.
DB_PATH = hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    repo_type="dataset",
    local_dir=local_dir
)

def get_connection():
    return sqlite3.connect(DB_PATH)

## 1. 데이터 로드

데이터베이스에서 `Station_Catchment_Buildings` 테이블과 `Stations` 테이블을 조인하여 데이터를 가져옵니다.

In [ ]:
conn = get_connection()

query = """
SELECT 
    b.*, 
    s.station_name_kr
FROM Station_Catchment_Buildings b
JOIN Stations s ON b.station_id = s.station_id
"""

df = pd.read_sql(query, conn)
conn.close()

print(f"Total Rows: {len(df)}")
display(df.head())

## 2. 기본 정보 확인

데이터 타입과 결측치를 확인합니다.

In [ ]:
df.info()

In [ ]:
display(df.describe())

### 결측치 시각화

In [ ]:
missing_counts = df.isnull().sum()
missing_percent = (missing_counts / len(df)) * 100

fig = go.Figure(data=[go.Bar(
    x=missing_percent.index,
    y=missing_percent.values,
    text=missing_percent.values.round(1),
    textposition='auto'
)])

fig.update_layout(
    title="컬럼별 결측치 비율 (%)",
    xaxis_title="컬럼",
    yaxis_title="결측치 비율 (%)",
    yaxis_range=[0, 100]
)

fig.show()

## 3. 단변량 분석 (Univariate Analysis)

### 건물 주용도 (Usage Type) 분포

In [ ]:
usage_counts = df['usage_type'].value_counts().head(20)

fig = go.Figure(data=[go.Bar(
    x=usage_counts.index,
    y=usage_counts.values,
    marker_color='royalblue'
)])

fig.update_layout(
    title="건물 주용도 상위 20개 분포",
    xaxis_title="주용도",
    yaxis_title="건물 수",
    xaxis_tickangle=-45
)

fig.show()

### 건물 구조 (Structure Type) 분포

In [ ]:
structure_counts = df['structure_type'].value_counts().head(20)

fig = go.Figure(data=[go.Bar(
    x=structure_counts.index,
    y=structure_counts.values,
    marker_color='crimson'
)])

fig.update_layout(
    title="건물 구조 상위 20개 분포",
    xaxis_title="구조",
    yaxis_title="건물 수",
    xaxis_tickangle=-45
)

fig.show()

### 건물 높이 (Height) 분포
높이가 0이거나 결측치인 경우는 제외하고 시각화합니다.

In [ ]:
height_data = df[df['height'] > 0]['height']

fig = go.Figure(data=[go.Histogram(
    x=height_data,
    nbinsx=100,
    marker_color='green'
)])

fig.update_layout(
    title="건물 높이 분포 (0m 초과)",
    xaxis_title="높이 (m)",
    yaxis_title="빈도",
    xaxis_range=[0, 100] # 대부분의 건물이 저층일 수 있으므로 줌인
)

fig.show()

### 연면적 (Floor Area) 분포

In [ ]:
area_data = df[df['floor_area'] > 0]['floor_area']

fig = go.Figure(data=[go.Histogram(
    x=area_data,
    nbinsx=200,
    marker_color='orange'
)])

fig.update_layout(
    title="건물 연면적 분포 (0 초과)",
    xaxis_title="연면적 (m²)",
    yaxis_title="빈도",
    xaxis_type="log" # 편차가 크므로 로그 스케일 사용 고려
)

fig.show()

## 4. 이변량 분석 (Bivariate Analysis)

### 역별 건물 수 Top 20

In [ ]:
station_counts = df['station_name_kr'].value_counts().head(20)

fig = go.Figure(data=[go.Bar(
    x=station_counts.index,
    y=station_counts.values,
    marker_color='purple'
)])

fig.update_layout(
    title="역세권 내 건물 수가 많은 역 Top 20",
    xaxis_title="역명",
    yaxis_title="건물 수",
    xaxis_tickangle=-45
)

fig.show()

### 역별 평균 건물 높이 Top 20

In [ ]:
station_avg_height = df[df['height'] > 0].groupby('station_name_kr')['height'].mean().sort_values(ascending=False).head(20)

fig = go.Figure(data=[go.Bar(
    x=station_avg_height.index,
    y=station_avg_height.values,
    marker_color='teal'
)])

fig.update_layout(
    title="역세권 평균 건물 높이가 높은 역 Top 20",
    xaxis_title="역명",
    yaxis_title="평균 높이 (m)",
    xaxis_tickangle=-45
)

fig.show()

## 3. 건물 주용도 (`usage_type`) 분석

역세권 내 건물의 주용도 분포를 분석합니다.

In [ ]:
usage_counts = df['usage_type'].value_counts().head(20)
display(usage_counts)

In [ ]:
fig = go.Figure(data=[go.Bar(
    x=usage_counts.index,
    y=usage_counts.values,
    marker_color='indianred'
)])

fig.update_layout(
    title="역세권 건물 주용도 Top 20",
    xaxis_title="용도",
    yaxis_title="건물 수",
    xaxis_tickangle=-45
)

fig.show()

## 4. 세대수 및 가구수 분석

역세권 내 건물의 세대수와 가구수 분포를 분석합니다.
세대수(Households)는 법적인 세대 단위를 의미하며, 가구수(Families)는 실제 거주하는 가구 단위를 의미할 수 있습니다 (데이터 정의에 따라 상이할 수 있음).

In [ ]:
print("세대수 기술통계:")
display(df['households'].describe())

print("\n가구수 기술통계:")
display(df['families'].describe())

In [ ]:
# 세대수가 0인 경우는 제외하고, 분포를 확인하기 위해 상위 1% 이상치 제거 후 시각화
valid_households = df[df['households'] > 0]
upper_limit = valid_households['households'].quantile(0.99)
filtered_households = valid_households[valid_households['households'] < upper_limit]

fig = go.Figure()
fig.add_trace(go.Histogram(x=filtered_households['households'], name='세대수', nbinsx=30))

fig.update_layout(
    title=f"건물별 세대수 분포 (상위 1% 제외, Max: {upper_limit})",
    xaxis_title="세대수",
    yaxis_title="빈도",
    bargap=0.1
)
fig.show()

In [ ]:
# 연면적과 세대수의 상관관계 (양의 상관관계 예상)
# 데이터가 많으므로 샘플링하여 시각화
sample_df = df[(df['floor_area'] > 0) & (df['households'] > 0)].sample(10000, random_state=42)

fig = go.Figure(data=go.Scatter(
    x=sample_df['floor_area'],
    y=sample_df['households'],
    mode='markers',
    marker=dict(
        size=5,
        color=sample_df['households'],
        colorscale='Viridis',
        showscale=True
    ),
    text=sample_df['building_name']
))

fig.update_layout(
    title="연면적 vs 세대수 (Sample 10,000)",
    xaxis_title="연면적 (m²)",
    yaxis_title="세대수",
    xaxis_type="log", # 로그 스케일 적용
    yaxis_type="log"
)
fig.show()